# Various approaches to load PDF documents

In [23]:
PDF_DOCUMENT="/data-transfer/iihf/230524_iihf_rulebook_gesamt_v_6_0_rz_v1.pdf"

## Using Langchain PyPDFLoader

!pip install pypdf

In [3]:
! ls /data-transfer/iihf

230524_iihf_rulebook_gesamt_v_6_0_rz_v1.pdf  rulebook.md


In [24]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(f"{PDF_DOCUMENT}")
pages = loader.load_and_split()

In [25]:
pages[0]

Document(page_content='INTERNATIONAL ICE HOCKEY FEDERATION\nIIHF OFFICIAL  \nRULEBOOK  \n2023 / 24  \nIIHF Official Rulebook 2023/24 – Version 1.0, May 2023', metadata={'source': '/data-transfer/iihf/230524_iihf_rulebook_gesamt_v_6_0_rz_v1.pdf', 'page': 0})

In [9]:
pages[1].page_content

'IIHF OFFICIAL RULE BOOK 2023/24 2\nWELCOME\n \n  \n  \n  \n \n \nIt is, therefore, important to make a clear separation between the purpose of all the elements of the \ngame and to use these respectfully. These distinctions can be taught at an early age or whenever \none begins to show interest in the game. And this is why hockey development begins with parents \nand coaches, those people most influential in guiding a person, old or young, into playing the game \nproperly and within the rules.\nThe IIHF Championship program encompasses 81 Member National Associations, five age and  \ngender categories over 30 international ice hockey tournaments, including the Olympic Winter \nGames. \nThe extensiveness of the program is acknowledged in the rule book. The goal is to provide everyone \none set of rules from which to work. This presents a fair and leveled standard of play. It is a means \nof keeping the game’s “language” the same regardless of where it is played.\nINTERNATIONAL ICE HOCK

In [16]:
pages[25].page_content

"TABLE OF CONTENTS3.2. PENALTY BOX\nEach Rink must be provided with benches or seats to be known as the “Penalty Box”. \nSeparate Penalty Boxes shall be provided for each Team and they shall be situated on the opposite sides directly across the ice from \ntheir Players' Benches. Teams must use the “Penalty Box” opposite their Players’ Bench and must use the same “Penalty Box” for \nthe duration of a game.\nEach “Penalty Box” should be at least 4.0 m in length and 1.50 m in width and shall be separated from the spectators by a Protective \nGlass to afford the necessary protection for the Players. \nEach Penalty Box must be of the same size and quality, offering no advantage to either Team in any manner and must have only one \ndoor for both entry and exit and must be operated only by the “Penalty Box Attendant”.\nOnly the Penalty Box Attendant, penalized Skaters, and Game Officials are allowed access to the Penalty Boxes.\n ➔For more information refer to Appendix VI – Infographics\nRULE

## Using LLMSherpa

## Using Unstructured.IO

## Using LlamaParse

In [ ]:
!pip3 install llama-index
!pip3 install llama-index-core
!pip3 install llama-index-embeddings-openai
!pip3 install llama-parse
!pip3 install llama-index-llms-ollama
!pip3 install ollama

In [36]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

import os
# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-dSMKpwu0fPsVOHfM3jxSsOYuMhS4ramkq9G1wjpQrHLZk9ny"

In [37]:
from llama_parse import LlamaParse

pdf_file_name = f"{PDF_DOCUMENT}"

documents = LlamaParse(result_type="markdown").load_data(pdf_file_name)


Started parsing the file under job_id 2da3ceb0-27d1-40c6-a589-f4e4b5c2881b


In [63]:
# Check loaded documents

print(f"Number of documents: {len(documents)}")

for doc in documents:
    print(doc.doc_id)
    print(doc.text[26000:30000] + '...')

Number of documents: 1
af97ad3c-82e6-4c9e-8074-a8f39dcecf56
 AREA

RULE 3 BENCHES

3.1. PLAYERS’ BENCHES

Each Rink shall be provided with seats or benches for the use by Players of both Teams. The only people allowed on or at the Players’ Benches are the dressed Players and not more than eight (8) persons, including Coach and team personnel. Teams must use the same Player’s Bench for the duration of a game. The accommodations provided, including benches and doors, must be uniform for both Teams.

The Players’ Benches shall be placed immediately alongside the ice as near to the center of the Rink as possible. Two (2) doors for each Players’ Bench must be uniform in location and size and as conveniently close to the Dressing Rooms as possible. Each Players’ Bench should be 10 m in length and 1.50 m in width and when situated in the spectator area, shall be separated from the spectators by a Protective Glass to afford the necessary protection for the Players and Team Personnel.

The Play

In [39]:
from llama_index.llms.openai import OpenAI
#from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core import  VectorStoreIndex
from llama_index.core import Settings

# Using OpenAI API for embeddings/llms
os.environ["OPENAI_API_KEY"] = "N.A. we use ollama"

EMBEDDING_MODEL  = "mxbai-embed-large"
GENERATION_MODEL = "llama3:8b"

#llm = OpenAI(model=GENERATION_MODEL)
llm = Ollama(model=GENERATION_MODEL, base_url="http://192.168.1.102:11434", request_timeout=120.0)

Settings.llm = llm


In [40]:
# Parse the documents using MarkdownElementNodeParser

from llama_index.core.node_parser import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(llm=llm, num_workers=3)

nodes = node_parser.get_nodes_from_documents(documents)


100it [00:00, 145990.39it/s]

100%|██████████| 100/100 [14:14<00:00,  8.54s/it]


In [41]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

print(f"Number of nodes: {len(base_nodes)}")

Number of nodes: 183


In [64]:
base_nodes[25].text

'---\n TEAMS\n\nIf a Player is assessed a penalty, and during the penalty they are found to be an ineligible Player, they will be removed from the game and another Player of their Team, designated by the Coach, must serve the remainder of the penalty. No additional penalties are to be assessed but a report of the incident must be submitted to the Proper Authorities.\n\nFor more information refer to IIHF Sport Regulations and IIHF Statutes & Bylaws.\n\n 5.3. GOALKEEPER\n\nEach Team shall be allowed one (1) Goalkeeper on the ice at one time. The Goalkeeper may be removed and substituted by another Skater. Such substitute shall not be permitted the privileges of the Goalkeeper. Each Team shall have on its Players’ Bench, or on a chair immediately beside the Players’ Bench (or nearby), a substitute Goalkeeper who shall always be fully equipped and ready to play.\n\nEach team shall have two (2) Goalkeepers listed on the Official Game Sheet. Exceptions to this rule are permitted in specific 